In [1]:
# %%
%pip install transformers datasets wandb accelerate

# %%
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import numpy as np
import os
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found 

2025-05-18 09:02:46.611601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747558967.068657      71 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747558967.221108      71 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:

# %%
# Set up environment variables for authentication
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_TOKEN")
assert(os.getenv('HF_TOKEN'))
assert(os.getenv('WANDB_API_KEY'))


In [3]:

# %%
# Create a unique run name based on timestamp
from datetime import datetime
import pytz

now_utc = datetime.now(pytz.utc)
now_colombo = now_utc.astimezone(pytz.timezone('Asia/Colombo'))
time_str = now_colombo.strftime('%Y-%b-%d_%H-%M-%S')
run_name = f'full-ft-{time_str}'
print(run_name)


full-ft-2025-May-18_14-33-09


In [4]:

# %%
# Initialize wandb and login to HuggingFace
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="choreo-doc-full-ft", name=run_name)

login(token=os.getenv('HF_TOKEN'))


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:

# %%
# First set the default device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [6]:

# Load tokenizer and model
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [7]:

# Load model with explicit device placement to avoid mixed-device issues
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
)

# Explicitly move model to the device
model = model.to(device)


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [8]:

# Make sure all parameters are trainable (unlike LoRA which only trains a subset)
for param in model.parameters():
    param.requires_grad = True

# %%
# Print trainable parameters info
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}% of total)")


Trainable parameters: 494,032,768 (100.00% of total)


In [9]:

# Check that all parameters are on the correct device
device_check = all(p.device == device for p in model.parameters())
print(f"All parameters on {device}: {device_check}")


All parameters on cuda: False


In [10]:

# %%
# Load dataset
dataset = load_dataset("json", data_files="/kaggle/input/choreo-dataset/choreo_dataset.jsonl")


Generating train split: 0 examples [00:00, ? examples/s]

In [11]:

# %%
# Split the dataset into training and validation sets
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(f"Training examples: {len(train_dataset)}, Evaluation examples: {len(eval_dataset)}")


Training examples: 607, Evaluation examples: 68


In [12]:

# %%
# Preprocess and tokenize dataset
def preprocess_function(examples):
    # Format conversations using the model's chat template
    conversations = []
    
    for instruction, inp, out in zip(examples['instruction'], examples['input'], examples['output']):
        # Create conversation with system instruction, user input, and assistant output
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": inp},
            {"role": "assistant", "content": out}
        ]
        conversations.append(messages)
    
    # Apply the model's built-in chat template
    formatted_texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=False) 
                      for conv in conversations]
    
    # Tokenize the formatted texts
    tokenized = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )
    
    # Create labels (for causal LM, typically identical to input_ids)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized


In [13]:

# %%
# Apply preprocessing to datasets
print("Preprocessing training dataset...")
tokenized_train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

print("Preprocessing evaluation dataset...")
tokenized_eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)


Preprocessing training dataset...


Map:   0%|          | 0/607 [00:00<?, ? examples/s]

Preprocessing evaluation dataset...


Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [14]:

# %%
# Determine whether to use fp16 or bf16 based on hardware support
bf16_available = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
print(f"BF16 support available: {bf16_available}")


BF16 support available: True


In [15]:

# Set up training arguments - simplified without DeepSpeed
training_args = TrainingArguments(
    output_dir="./qwen_choreo_full_ft",
    learning_rate=1e-5,  # Slightly higher learning rate since we're not using DeepSpeed
    per_device_train_batch_size=1,  # Very small batch size to avoid OOM errors
    gradient_accumulation_steps=16,  # Increased accumulation to compensate for smaller batch
    per_device_eval_batch_size=1,
    num_train_epochs=3,  # Reduced epochs for faster training
    weight_decay=0.01,
    logging_steps=1,
    logging_first_step=True,
    eval_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=20,
    # Precision options
    bf16=bf16_available,  # Use BF16 if supported
    fp16=not bf16_available and torch.cuda.is_available(),  # Otherwise use FP16 if on CUDA
    logging_dir="./logs",
    load_best_model_at_end=True,
    save_total_limit=4,
    report_to="wandb",
    push_to_hub=True,
    hub_model_id=f"qwen-choreo-full-ft-{time_str}",
    run_name=run_name,
    # Memory optimization
    gradient_checkpointing=True,
    optim="adamw_torch",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    # Avoid device issues
    no_cuda=not torch.cuda.is_available(),  # Only disable CUDA if not available
    dataloader_num_workers=1,  # Reduced for stability
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
)


/tmp/ipykernel_71/4183412951.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
5,3.371400,5.053649
10,0.354600,0.740638
15,0.297500,0.659138
20,0.290800,0.610927
25,0.276800,0.589317
30,0.295700,0.577857
35,0.266000,0.570947
40,0.205000,0.567414
45,0.293900,0.565714
50,0.290300,0.565357


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to 

TrainOutput(global_step=57, training_loss=0.7154842638655713, metrics={'train_runtime': 2484.0522, 'train_samples_per_second': 0.733, 'train_steps_per_second': 0.023, 'total_flos': 4004244246233088.0, 'train_loss': 0.7154842638655713, 'epoch': 3.0})

In [17]:
# Save the model locally
print("Saving model...")
model.save_pretrained("./qwen_choreo_full_ft_final")
tokenizer.save_pretrained("./qwen_choreo_full_ft_final")

print("Pushing model to Hub...")
model.push_to_hub(f"qwen-choreo-full-ft-{time_str}-final")
tokenizer.push_to_hub(f"qwen-choreo-full-ft-{time_str}-final")


Saving model...
Pushing model to Hub...


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rtweera/qwen-choreo-full-ft-2025-May-18_14-33-09-final/commit/72ff706d3ab8e529f89f02cfdaf310b8e8ecc39b', commit_message='Upload tokenizer', commit_description='', oid='72ff706d3ab8e529f89f02cfdaf310b8e8ecc39b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rtweera/qwen-choreo-full-ft-2025-May-18_14-33-09-final', endpoint='https://huggingface.co', repo_type='model', repo_id='rtweera/qwen-choreo-full-ft-2025-May-18_14-33-09-final'), pr_revision=None, pr_num=None)

In [18]:
# Test the model with a sample
print("Testing the fine-tuned model...")
test_input = "Whart is choreo?"

# Create conversation with the test input
test_messages = [
    {"role": "user", "content": test_input}
]
formatted_test = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_test, return_tensors="pt").to(device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print("Model response:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
DynamicCache + torch.export is tested on torch 2.6.0+ and may not work on earlier versions.


Testing the fine-tuned model...
Model response:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Whart is choreo?<|im_end|>
<|im_start|>assistant
Choreo is an open-source, high-performance data processing framework developed and maintained by Alibaba Cloud. It provides a comprehensive set of tools for building scalable, efficient, and flexible data pipelines in various domains such as machine learning, natural language processing, IoT, and more.\n\nChoreo supports multiple programming languages including Python, Java, C++, and Go. With Choreo, you can easily build applications that process large volumes of data efficiently.\n\nKey features of Choreo include:\n- **High Performance**: Designed to handle massive datasets with low latency.\n- **Scalability**: Automatically scales your application based on the load and performance requirements.\n- **Flexibility**: Supports both batch and stream operations.\n- **Security**: Bu